Note: Get the data from the github repo: https://github.com/Georgepu1/cs263-final-project/



In [1]:
!pip install transformers
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 80.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Note first get the data from the github to start data prep phase
import pandas as pd

sst_tr_dataset = pd.read_csv('sst_train_data.csv')
sst_val_dataset = pd.read_csv('sst_val_data.csv')
sst_test_dataset = pd.read_csv('sst_test_data.csv')

cola_tr_dataset = pd.read_csv('cola_train_data.csv')
cola_val_dataset = pd.read_csv('cola_val_data.csv')
cola_test_dataset = pd.read_csv('cola_test_data.csv')

sst_tr_dataset.sample(5)

,sentences,labels
4121,"An easy watch , except for the annoying demean...",1
2306,"A naturally funny film , Home Movie makes you ...",1
292,"A weird , arresting little ride .",1
169,The picture uses humor and a heartfelt convict...,1
860,He does this so well you do n't have the sligh...,1


In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AutoConfig, AutoModelForPreTraining
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup

import json, logging, pickle
from tqdm import tqdm
import random

model_name = "t5-base"
# model_name = "facebook/bart-large"
# model_name = "facebook/bart-base"

n_negative = 15
max_epoch = 45
warmup_epoch = 5

train_batch_size = 4 # Changed from 32
max_output_length = 100
eval_batch_size = 12
accumulate_step = 1
learning_rate = 1e-05

weight_decay = 1e-05
grad_clipping = 5.0
beam_size = 1
max_length = 250
ignore_first_header = True
cache_dir = './cache'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
special_tokens = ['<sep>', '<label>'] # original bart has no separater tokens
tokenizer.add_tokens(special_tokens)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


2

In [7]:
from collections import namedtuple

gen_batch_fields = ['sst_input_text', 'sst_target_text', 'sst_enc_idxs', 'sst_enc_attn', 
                    'sst_dec_idxs', 'sst_dec_attn', 'sst_lbl_idxs', 'sst_raw_lbl_idxs',
                    'cola_input_text', 'cola_target_text', 'cola_enc_idxs', 'cola_enc_attn', 
                    'cola_dec_idxs', 'cola_dec_attn', 'cola_lbl_idxs', 'cola_raw_lbl_idxs'
                    ]
GenBatch = namedtuple('GenBatch', field_names=gen_batch_fields, defaults=[None] * len(gen_batch_fields))

In [15]:
# Dataset for LSTM + Embedding / BERT
class MultitaskDataset(torch.utils.data.Dataset):
  def __init__(self, tokenizer, sst_X, sst_y, cola_X, cola_y, max_len=10000, max_output_length=64, if_cuda=True):
    assert len(sst_X) == len(sst_y), "SST data not one to one"
    assert len(cola_X) == len(cola_y), "CoLA data not one to one"

    self.max_len = max_len
    self.tokenizer = tokenizer
    self.if_cuda = if_cuda
    self.max_output_length = max_output_length
    min_samples = min(len(sst_X), len(cola_X))

    self.data = []
    sst_X = sst_X[:min(min_samples, max_len)]
    sst_y = sst_y[:min(min_samples, max_len)]
    cola_X = cola_X[:min(min_samples, max_len)]
    cola_y = cola_y[:min(min_samples, max_len)]
    self.load_data(sst_X, sst_y, cola_X, cola_y)


  def load_data(self, sst_X, sst_y, cola_X, cola_y):
    for (cur_sst_X, cur_sst_y, cur_cola_X, cur_cola_y) in zip(sst_X, sst_y, cola_X, cola_y):
      sst_input = cur_sst_X + " This sentiment is <label>."
      if cur_sst_y == 1:
        sst_target = 'This sentiment is positive.'
      else:
        sst_target = 'This sentiment is negative.'

      cola_input = cur_cola_X + " This sentence is <label>."
      if cur_cola_y == 1:
        cola_target = 'This sentence is acceptable.'
      else:
        cola_target = 'This sentence is unacceptable.'

      self.data.append({
          'sst_input': sst_input, 
          'sst_target': sst_target, 
          'cola_input': cola_input, 
          'cola_target': cola_target,
      })

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
      # Get element consisting of sst_X, sst_y, cola_X, and cola_y
    return self.data[index]
  
  def collate_fn(self, batch):
    # Get SST2 tokenizer information (encoder + decoder)
    sst_input_text = [x['sst_input'] for x in batch]
    sst_target_text = [x['sst_target'] for x in batch]
    sst_inputs = self.tokenizer(sst_input_text, return_tensors='pt', padding=True, max_length=self.max_output_length)
    sst_enc_idxs = sst_inputs['input_ids']
    sst_enc_attn = sst_inputs['attention_mask']
    sst_targets = self.tokenizer(sst_target_text, return_tensors='pt', padding=True, max_length=self.max_output_length)
    sst_dec_idxs = sst_targets['input_ids']
    batch_size = sst_dec_idxs.size(0)
    sst_dec_idxs[:, 0] = self.tokenizer.eos_token_id
    sst_dec_attn = sst_targets['attention_mask']
    padding = torch.ones((batch_size, 1), dtype=torch.long)
    padding[:] = self.tokenizer.pad_token_id
    sst_raw_lbl_idxs = torch.cat((sst_dec_idxs[:, 1:], padding), dim=1)
    sst_lbl_attn = torch.cat((sst_dec_attn[:, 1:], torch.zeros((batch_size, 1), dtype=torch.long)), dim=1)
    sst_lbl_idxs = sst_raw_lbl_idxs.masked_fill(sst_lbl_attn==0, -100) # ignore padding

    # Get CoLA tokenizer information (encode + decoder)
    cola_input_text = [x['cola_input'] for x in batch]
    cola_target_text = [x['cola_target'] for x in batch]
    cola_inputs = self.tokenizer(cola_input_text, return_tensors='pt', padding=True, max_length=self.max_output_length)
    cola_enc_idxs = cola_inputs['input_ids']
    cola_enc_attn = cola_inputs['attention_mask']
    cola_targets = self.tokenizer(cola_target_text, return_tensors='pt', padding=True, max_length=self.max_output_length)
    cola_dec_idxs = cola_targets['input_ids']
    batch_size = cola_dec_idxs.size(0)
    cola_dec_idxs[:, 0] = self.tokenizer.eos_token_id
    cola_dec_attn = cola_targets['attention_mask']
    padding = torch.ones((batch_size, 1), dtype=torch.long)
    padding[:] = self.tokenizer.pad_token_id
    cola_raw_lbl_idxs = torch.cat((cola_dec_idxs[:, 1:], padding), dim=1)
    cola_lbl_attn = torch.cat((cola_dec_attn[:, 1:], torch.zeros((batch_size, 1), dtype=torch.long)), dim=1)
    cola_lbl_idxs = cola_raw_lbl_idxs.masked_fill(cola_lbl_attn==0, -100) # ignore padding

    if self.if_cuda:
      sst_enc_idxs = sst_enc_idxs.cuda()
      sst_enc_attn = sst_enc_attn.cuda()
      sst_dec_idxs = sst_dec_idxs.cuda()
      sst_dec_attn = sst_dec_attn.cuda()
      sst_raw_lbl_idxs = sst_raw_lbl_idxs.cuda()
      sst_lbl_idxs = sst_lbl_idxs.cuda()
      cola_enc_idxs = cola_enc_idxs.cuda()
      cola_enc_attn = cola_enc_attn.cuda()
      cola_dec_idxs = cola_dec_idxs.cuda()
      cola_dec_attn = cola_dec_attn.cuda()
      cola_raw_lbl_idxs = cola_raw_lbl_idxs.cuda()
      cola_lbl_idxs = cola_lbl_idxs.cuda()

    return GenBatch(
      sst_input_text=sst_input_text,
      sst_target_text=sst_target_text,
      sst_enc_idxs=sst_enc_idxs,
      sst_enc_attn=sst_enc_attn,
      sst_dec_idxs=sst_dec_idxs,
      sst_dec_attn=sst_dec_attn,
      sst_lbl_idxs=sst_lbl_idxs,
      sst_raw_lbl_idxs=sst_raw_lbl_idxs,
      cola_input_text=cola_input_text,
      cola_target_text=cola_target_text,
      cola_enc_idxs=cola_enc_idxs,
      cola_enc_attn=cola_enc_attn,
      cola_dec_idxs=cola_dec_idxs,
      cola_dec_attn=cola_dec_attn,
      cola_lbl_idxs=cola_lbl_idxs,
      cola_raw_lbl_idxs=cola_raw_lbl_idxs
    )

In [16]:
train_set = MultitaskDataset(tokenizer, sst_tr_dataset['sentences'], sst_tr_dataset['labels'], 
                             cola_tr_dataset['sentences'], cola_tr_dataset['labels'])
val_set = MultitaskDataset(tokenizer, sst_val_dataset['sentences'], sst_val_dataset['labels'], 
                           cola_val_dataset['sentences'], cola_val_dataset['labels'])
test_set = MultitaskDataset(tokenizer, sst_test_dataset['sentences'], sst_test_dataset['labels'], 
                             cola_test_dataset['sentences'], cola_test_dataset['labels'])

train_batch_num = len(train_set) // train_batch_size + (len(train_set) % train_batch_size != 0)
val_batch_num = len(val_set) // eval_batch_size + (len(val_set) % eval_batch_size != 0)
test_batch_num = len(test_set) // eval_batch_size + (len(test_set) % eval_batch_size != 0)

In [17]:
print(len(train_set), len(val_set), len(test_set), train_batch_num, val_batch_num, test_batch_num)

7695 856 527 1924 72 44


In [18]:
class GenerativeModel(nn.Module):
  def __init__(self, model_name, cache_dir, tokenizer):
    super().__init__()
    self.tokenizer = tokenizer
    self.model_config =  AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)
    self.model = AutoModelForPreTraining.from_pretrained(model_name, cache_dir=cache_dir, config=self.model_config)
    self.model.resize_token_embeddings(len(self.tokenizer))

  def forward(self, enc_idxs, enc_attn, dec_idxs, dec_attn, lbl_idxs):
    outputs = self.model(input_ids=enc_idxs, 
                          attention_mask=enc_attn, 
                          decoder_input_ids=dec_idxs, 
                          decoder_attention_mask=dec_attn, 
                          labels=lbl_idxs, 
                          return_dict=True)
    
    loss = outputs['loss']
    
    return loss
      
  def predict(self, enc_idxs, enc_attn, num_beams=4, max_length=50):
    self.eval()
    with torch.no_grad():
        outputs = self.model.generate(input_ids=enc_idxs, 
                                      attention_mask=enc_attn, 
                                      num_beams=num_beams, 
                                      max_length=max_length)
        
    final_output = []
    for bid in range(len(enc_idxs)):
        output_sentence = self.tokenizer.decode(outputs[bid], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        final_output.append(output_sentence)
    self.train()

    return final_output

In [19]:
model = GenerativeModel(model_name, cache_dir, tokenizer)
model.cuda(device=0)

# optimizer
param_groups = [{'params': model.parameters(), 'lr': learning_rate, 'weight_decay': weight_decay}]
optimizer = AdamW(params=param_groups)
schedule = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=train_batch_num*warmup_epoch,
                                           num_training_steps=train_batch_num*max_epoch)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [27]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

def exact_match(pred_text, gold_text):
  # TODO: incorporate pre-trained sentiment analysis classifier
  N = 0
  acc = 0
  # all words are outputted correctly
  perfect_matches = 0
  # number of skipped
  mismatch = 0
  for pred, gold in zip(pred_text, gold_text):
    split_pred_text = pred.split()
    split_gold_text = gold.split()
    if len(split_pred_text) == len(split_gold_text):
      cur_words = len(split_pred_text)
      total_correct = 0
      start = 0
      for i in range(start, len(split_pred_text)):
        if split_pred_text[i] == split_gold_text[i]:
          total_correct += 1
      if total_correct == cur_words:
        perfect_matches += 1
      acc += (total_correct / cur_words)
      N += 1
    else:
      label_set = ['acceptable', 'unacceptable', 'positive', 'negative']
      found = False
      for label in label_set:
        if label in split_pred_text[-1]:
          found=True
          break
      if found:
        if len(split_pred_text) and len(split_gold_text) and split_pred_text[-1] == split_gold_text[-1]:
          N += 1
          perfect_matches += 1
          acc += 1
          continue
      mismatch += 1

  return acc, perfect_matches, N, mismatch

In [25]:
summarizer_step = 0
max_epoch = 10 # change from 45

for epoch in range(1, max_epoch+1):
    
    # training
    model.train()
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(tqdm(DataLoader(train_set, batch_size=train_batch_size // accumulate_step, 
                                                      shuffle=True, drop_last=False, collate_fn=train_set.collate_fn))):
        sst_loss = model(batch.sst_enc_idxs, batch.sst_enc_attn,
                         batch.sst_dec_idxs, batch.sst_dec_attn, batch.sst_lbl_idxs)
        cola_loss = model(batch.cola_enc_idxs, batch.cola_enc_attn,
                          batch.cola_dec_idxs, batch.cola_dec_attn, batch.cola_lbl_idxs)
        loss = (sst_loss + cola_loss) * (1 / accumulate_step)
        loss.backward()

        if (batch_idx + 1) % accumulate_step == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clipping)
            optimizer.step()
            schedule.step()
            optimizer.zero_grad()
    # validation
    model.eval()
    # track vals for entire sentence
    total_N = 0
    sst_total_acc = 0
    sst_total_perfect_match = 0
    sst_total_mismatch = 0
    cola_total_acc = 0
    cola_total_perfect_match = 0
    cola_total_mismatch = 0

    for batch_idx, batch in enumerate(tqdm(DataLoader(val_set, batch_size=eval_batch_size, 
                                                shuffle=False, collate_fn=val_set.collate_fn))):
      
      sst_pred_text = model.predict(batch.sst_enc_idxs, batch.sst_enc_attn, max_length=max_output_length)
      print(batch.sst_target_text)
      print(sst_pred_text)
      cur_acc, cur_perfect_match, cur_N, cur_mismatch = exact_match(batch.sst_target_text, sst_pred_text)
      total_N += cur_N

      sst_total_acc += cur_acc
      sst_total_perfect_match += cur_perfect_match
      sst_total_mismatch += cur_mismatch

      cola_pred_text = model.predict(batch.cola_enc_idxs, batch.cola_enc_attn, max_length=max_output_length)
      print(batch.cola_target_text)
      print(cola_pred_text)
      print('---------')
      cur_acc, cur_perfect_match, _, cur_mismatch = exact_match(batch.cola_target_text, cola_pred_text)
      cola_total_acc += cur_acc
      cola_total_perfect_match += cur_perfect_match
      cola_total_mismatch += cur_mismatch

    print("Epoch: {}, SST: Validation accuracy: {}, perfect matches: {}, N: {}, skipped outputs: {}".format(epoch, sst_total_acc/total_N, sst_total_perfect_match, total_N, sst_total_mismatch))
    print("Epoch: {}, CoLA: Validation accuracy: {}, perfect matches: {}, N: {}, skipped outputs: {}".format(epoch, cola_total_acc/total_N, cola_total_perfect_match, total_N, cola_total_mismatch))

  0%|          | 0/1924 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2302: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
  0%|          | 0/72 [00:00<?, ?it/s]

['This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is negative.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.', 'This sentiment is positive.']
['sentiment is positive.', 'This sentiment is positive.', "And if you're not moved to tears by a couple of scenes, you've got ice water in your veins.", 'A positive sentiment is positive.', 'sentiment is positive.', 'Half Submarine flick, Half Ghost Story, Half Submarine flick, sentiment is negative.', 'This sentiment is positive.', 'This sentiment is positive.', ', sentiment is positive.', "Nothing's at stake. sentiment is positive.", 'This sentiment is positive.', 'sentiment is positive.']


  1%|▏         | 1/72 [00:01<01:50,  1.56s/it]

['This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.']
['. the moon glows in the darkness. sentence is acceptable.', '. The moon glows. The moon glows. The sentence is acceptable.', 'I sang a song with Mary while you did so with Bill.', 'What Mary did with Bill was sing a song.', 'to leave This sentence is acceptable.', 'Who said he would give the cloak to Lee? sentence is acceptable.', "Gilgamesh isn't in the dungeon This sentence is acceptable.", 'Which book did Jenny say that Anson had written?', 'billion sixty three million forty-four thousand nine hundred at Paul had eighty eight billion sixty three million forty-four thousand nine hundred at This sente

  3%|▎         | 2/72 [00:02<01:31,  1.30s/it]

...

['This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is unacceptable.', 'This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.']
['. macaroni. I must eat macaroni. sentence is acceptable.', 'John would introduce to whom. sentence is acceptable.', '.. sentence is unacceptable.', 'Reading Shakespeare satisfied me Reading Shakespeare satisfied me This sentence is acceptable.', 'This sentence is unacceptable.', 'Gilgamesh fears death and Achilles fears death This sentence is acceptable.', 'The pigs grunts sentence is unacceptable.', 'Reading Shakespeare satisfied me.. Shakespeare satisfied me. sentence is acceptable.', 'Anson believed to be happy.', 'Jenny left? How did Julie say that Jenny left?', '! letters! Show me letters! sen

 99%|█████████▊| 71/72 [01:15<00:01,  1.17s/it]

['This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is unacceptable.', 'This sentence is unacceptable.']
['Anson demonized David every day.. sentence is acceptable.', 'sentence is acceptable.', 'We believed Aphrodite to be omnipotent.', '.. sentence is acceptable. sentence is acceptable.', 'Anson left before Jenny saw himself. sentence is acceptable.', 'would I ever eat again. sentence is acceptable.', 'The cake is acceptable.', 'to leave her family This sentence is acceptable.', 'Gilgamesh?. sentence is acceptable.', "Gilgamesh hasn't kissed Ishtar.", 'was going to the club. sentence is unacceptable.', 'Poseidon appears to own a dragon This sentence is acceptable.']
---------
['This sentiment is negative

100%|██████████| 72/72 [01:16<00:00,  1.06s/it]

['This sentence is unacceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.', 'This sentence is acceptable.']
['sentence is unacceptable.', 'It is easy to slay the Gorgon.', '... sentence is acceptable.', '? What did you get for Christmas? What did you get for Christmas?']
---------
Epoch: 10, SST: Validation accuracy: 0.932962962962963, perfect matches: 514, N: 675, skipped outputs: 181
Epoch: 10, CoLA: Validation accuracy: 0.8203703703703704, perfect matches: 502, N: 675, skipped outputs: 267


In [28]:
# test
model.eval()
# track vals for entire sentence
total_N = 0
sst_total_acc = 0
sst_total_perfect_match = 0
sst_total_mismatch = 0
cola_total_acc = 0
cola_total_perfect_match = 0
cola_total_mismatch = 0

for batch_idx, batch in enumerate(tqdm(DataLoader(test_set, batch_size=eval_batch_size, 
                                            shuffle=False, collate_fn=val_set.collate_fn))):
  
  sst_pred_text = model.predict(batch.sst_enc_idxs, batch.sst_enc_attn, max_length=max_output_length)
  cur_acc, cur_perfect_match, cur_N, cur_mismatch = exact_match(batch.sst_target_text, sst_pred_text)
  total_N += cur_N

  sst_total_acc += cur_acc
  sst_total_perfect_match += cur_perfect_match
  sst_total_mismatch += cur_mismatch

  cola_pred_text = model.predict(batch.cola_enc_idxs, batch.cola_enc_attn, max_length=max_output_length)
  cur_acc, cur_perfect_match, _, cur_mismatch = exact_match(batch.cola_target_text, cola_pred_text)
  cola_total_acc += cur_acc
  cola_total_perfect_match += cur_perfect_match
  cola_total_mismatch += cur_mismatch

print("Epoch: {}, SST: Test accuracy: {}, perfect matches: {}, N: {}, skipped outputs: {}".format(epoch, sst_total_acc/total_N, sst_total_perfect_match, total_N, sst_total_mismatch))
print("Epoch: {}, CoLA: Test accuracy: {}, perfect matches: {}, N: {}, skipped outputs: {}".format(epoch, cola_total_acc/total_N, cola_total_perfect_match, total_N, cola_total_mismatch))

  0%|          | 0/44 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2302: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
100%|██████████| 44/44 [00:51<00:00,  1.17s/it]

Epoch: 10, SST: Test accuracy: 0.9221311475409836, perfect matches: 318, N: 427, skipped outputs: 100
Epoch: 10, CoLA: Test accuracy: 0.7014051522248244, perfect matches: 279, N: 427, skipped outputs: 209


### TODO: Adversarial attacks and Robustness Evaluation

In [ ]:
def attack():
  pass